# Tracking traffic on the Burke Gilman trail
This notebook will walk you through the process of exploring traffic patterns on the Burke Gilman trail, and asking and answering specific research questions using that data. The data we're using today comes from [data.seattle.gov](https://data.seattle.gov/) which is an online portal where the city of Seattle hosts publicly-available datasets. 

The exercises below will use the [Burke Gilman trail north of NE 70th St Bicycle and Pedestrian Counter](https://data.seattle.gov/Transportation/Burke-Gilman-Trail-north-of-NE-70th-St-Bicycle-and/2z5v-ecg8) dataset, which collects north and south traffic by bicyclists and pedestrians on a high-traffic portion of the Burke Gilman trail. 

# Gathering the data
Our first step here is to gather a sample of the data to work with. The full dataset is 53,000 rows, but we can use an API query to specify what time range we want to look at, so that we don't need to download the entire dataset. For now, we'll grab one year's worth of data—2019. 

If you want more practice with the Seattle Open Data API, you can use the notebook ``SODA_API_demo.ipynb``. 

In [ ]:
#download the requests library, a bundle of code that is useful for sending and retrieving data over the internet
import requests

In [ ]:
#the URL we're retrieving the data from. Copy/paste it into your browser to view it!
api_endpoint = "https://data.seattle.gov/resource/2z5v-ecg8.json?"

#the parameters we're passing to the API, to specify what subset of data we want.
api_parameters = "$limit=50000&$where=date > '2019-01-01T00:00:00' AND date < '2020-01-01T00:00:00'"

Now we will combine these two strings into a single long URL, which is our API request. It ends up looking like this (run the cell below).

In [ ]:
print(api_endpoint + api_parameters)

Copy and paste that full string into your browser to get a sneak peak at the data we'll be working with!

In [ ]:
#request the data from the API
api_request = requests.get(api_endpoint + api_parameters)

#turn this data into a list of dictionaries, so that we can work with it in Python
raw_data = api_request.json()

Now we have the data, let's see what it looks like...

In [ ]:
#how many rows are in our dataset?
print(len(raw_data))

In [ ]:
print(raw_data)

That's kind of hard to read, isn't it? Fortunately, there's another useful Python library called "pretty print" (``pprint``), that we can use to make it easier to read this data structure.

In [ ]:
#import the pretty print library
from pprint import pprint 

You use ``pprint`` the same way you use ``print``

In [ ]:
pprint(raw_data)

Now that we have our data sample and we know how the data is organized (a list of dictionaries, with one dictionary for each hour of traffic), we can start asking research questions about the data.

## Question #1: how many people used the Burke Gilman during commute hours in 2019?

City planners often need to know the traffic volumes on particular roads and trails, so that they can prioritize maintenance and improvements. Anyone who's been on the Burke Gilman on a weekday at 5pm knows that it's a busy thoroughfare, but how busy is it?

For our first step, we need to decide what we mean by commute hours. Let's say between 6am and 9:59am is the "morning" communte, and 3pm-6:59pm is the "evening commute".

We'll store these values in two lists, so that as we loop through each hour in the dataset later we can check whether the traffic during that hour falls within the morning or evening commute. 

In [ ]:
morning_commute_hours = ['06:00:00','07:00:00', '08:00:00', '09:00:00']
evening_commute_hours = ['15:00:00','16:00:00', '17:00:00', '18:00:00']

Now we need some place to store the morning/evening commute counts, as we loop through the dataset. We'll create a dictionary with keys for 'morning' and 'evening' for this purpose. The values for each of these will start at 0, and increase as we loop through the dataset and add the traffic we see.

In [ ]:
#create an empty dictionary to hold our counts
commuters = {'morning':0, 'evening':0}

Now we can loop through our ``raw data`` list and examine each dictionary in that list. If the time of day in that dictionary matches one of the times we've listed in our morning and evening commute hour lists we made above, then we take the value of ``bgt_north_of_ne_70th_total`` for that dictionary and add it to one of the totals in ``commuters``. If it doesn't match, we move on to the next one and do the same thing.

In [ ]:
for hour_count in raw_data:

    traffic_hour = hour_count['date'][11:19] #get the timestamp
    
    if traffic_hour in morning_commute_hours: #if it's in the morning, add the count to our morning total
        commuters['morning'] = commuters['morning'] + int(hour_count['bgt_north_of_ne_70th_total'])
    
    elif traffic_hour in evening_commute_hours: #if it's in the evening, add the count to our evening total
        commuters['evening'] = commuters['evening'] + int(hour_count['bgt_north_of_ne_70th_total'])
    
    else:
        pass #we don't care who's on the BGT during non-commute hours, so we ignore it and move on

### Understanding the loop we just made

#### interlude 1: data types
If you scroll up to our pprinted dataset above, you'll see that the numeric values for ``bgt_north_of_ne_70th_total`` (the hourly traffic counts) have '' around them. This means they are being stored as strings.


In [ ]:
#we can ask Python to tell us what type of value this is using 'type'
print(type(raw_data[0]['bgt_north_of_ne_70th_total']))

If we want to add these numeric values together, we will need to turn them into integers. That's what's happening when you see ``int(hour_count['bgt_north_of_ne_70th_total'])`` in the loop above.

In [ ]:
#converting the string to an integer
str_to_int = int(raw_data[0]['bgt_north_of_ne_70th_total'])

print(type(str_to_int))

#### interlude 2: slicing
The date and time for each item in the list is stored in a single string, like ``'2019-01-01T06:00:00.000'`` Right now, we are only interested in the time of day, which is in the middle—between "T" (position 10) and "." (position 19), so we slice the beginning and the end of the string off before we compare the value with our commute hour lists.

In [ ]:
print(raw_data[0]['date'])

In [ ]:
print(raw_data[0]['date'][:11]) #slice the string to just get the stuff before the time

In [ ]:
print(raw_data[0]['date'][19:]) #slice the string to just get the stuff after the time

In [ ]:
print(raw_data[0]['date'][11:19]) #slice the string to just get the time

### Getting the answer to Q1
Now if our loop above worked, we'll know how many people used the Burke Gilman during commute hours in 2019 (at 70th street, at least)!

In [ ]:
pprint(commuters)

In [ ]:
#we can make these values into a human readable sentence, if we convert the ints back into strings
print("There were " 
    + str(commuters['morning']) 
    + " morning commuters and " 
    + str(commuters['evening']) 
    + " on the Burke Gilman at 70th Street in 2019!"
     )
      
      

## Question 2: what were the busiest hours on the Burke Gilman in 2019?
Let's say you want to know when the Burke Gilman is least likely to be congested, so you can plan your trip. How would you find this out? 

To do this, we need to start with a list of all of the hours of the day. We could do this manually by creating a list like we did with the morning and evening commute hours lists above, but since there are 24 hours in a day that would take a long time to type out manually! Is there a quicker way?

In [ ]:
#create an empty dictionary, that we will fill with hours
traffic_by_hour = {}

In [ ]:
#loop through the dataset again...
for hour_data in raw_data:
    
    traffic_hour = hour_data['date'][11:19] #grab the hour value from the item
    
    #if haven't seen this hour yet during the loop...
    if traffic_hour not in traffic_by_hour.keys():
        
        # ...create an item for it in our dictionary that we can use later to track our cumulative hourly counts
        traffic_by_hour[traffic_hour] = 0 #e.g {'01:00:00' : 0}
   
    else:
        pass
        #if we've already seen this hour and stored it in our dictionary, ignore it and move on to the next one

In [ ]:
#how many hours in our list?
len(traffic_by_hour)

In [ ]:
#what does our list look like?
pprint(traffic_by_hour)

## Getting the answer to Q2
Awesome! Now that we have buckets ready to hold counts for each hour in the day, we can loop through the raw data again and start calculating the traffic counts for each hour.

In [ ]:
for hour_data in raw_data:
    
    traffic_hour = hour_data['date'][11:19]

    traffic_by_hour[traffic_hour] =  traffic_by_hour[traffic_hour] + int(hour_data['bgt_north_of_ne_70th_total'])

    #you can also un-comment the following line and run it instead of the line directly above.
    #this is a slightly shorter way to do the same thing. Re-set traffic_by_hour to zero and try it!
#     traffic_by_hour[traffic_hour] += int(hour_data['bgt_north_of_ne_70th_total'])

In [ ]:
pprint(traffic_by_hour)

It looks like the busiest hour is between 5 and 6pm!

#### interlude 3: plus-equals
Writing long lines of code like ``commuters['morning'] = commuters['morning'] + int(hour_count['bgt_north_of_ne_70th_total'])`` can be tedious, and it makes it more likely that you will make a typo that causes your code to crash. There's an easier way!

Instead of writing ``thing_one = thing_one + thing_two``, you can write ``thing_one += thing_two``. It does the same thing! 

You can try this out yourself by re-running cells 35-38 above, and then using the code ``traffic_by_hour[traffic_hour] += int(hour_data['bgt_north_of_ne_70th_total'])`` 

...instead of the line ``traffic_by_hour[traffic_hour] =  traffic_by_hour[traffic_hour] + int(hour_data['bgt_north_of_ne_70th_total'])``

### Going further with Question 3
- Challenge #1: How would you use Python to find the HOUR with the most traffic? This list is short, so we can probably easily identify the element with the most traffic just by looking at it. But what if it had 10k items in it? How would YOU find the hour with the most traffic?
- Challegen #2: How would you find the busiest DAY on the Burke Gilman in 2019? You should be able to answer this question with the same techniques you used to find the busiest hour!


## Question 3: What is the busiest hours for bikes vs pedestrians?
One cool thing about this dataset is that it doesn't just count by hour, it counts by what kind of traffic: bikes or pedestrians. Let's run the same set of commands as above, but this time we'll break things apart into bikes and peds.

In [ ]:
#create an empty dictionary
traffic_by_hour_bp = {}

for hour_data in raw_data:
    
    traffic_hour = hour_data['date'][11:19]
    
    #if haven't seen this hour yet, create an item for it that we can use to track hourly counts later
    if traffic_hour not in traffic_by_hour_bp.keys():
        
        #this time, we create a dictionary-in-a-dictionary, to hold our bike and pedestrian counts separately
        traffic_by_hour_bp[traffic_hour] = {'bikes' : 0, 'pedestrians' : 0}
    
    else:
        pass
        #if we've already seen this hour and stored it in our dictionary, ignore it and move on

pprint(traffic_by_hour_bp)

## Getting the answer to Q3
Now that we have the right buckets for every value we want to capture, we're ready to get counts for bikes and peds separately.

In [ ]:
for hour_data in raw_data:
    
    traffic_hour = hour_data['date'][11:19]
    
    #note that we're using the 'plus-equals' technique now, which means we have to write less code
    traffic_by_hour_bp[traffic_hour]['bikes'] += int(hour_data['bike_north']) #add northbound bike counts
    traffic_by_hour_bp[traffic_hour]['bikes'] += int(hour_data['bike_south']) #add southbound bike counts
    
    traffic_by_hour_bp[traffic_hour]['pedestrians'] += int(hour_data['ped_north']) #add northbound ped counts
    traffic_by_hour_bp[traffic_hour]['pedestrians'] += int(hour_data['ped_south']) #add sounthbound ped counts

In [ ]:
pprint(traffic_by_hour_bp)

Now that there is more data in the notebook, it's a bit harder to figure out the busiest hour for bikes and pedestrians separately, just by looking. So let's use a quick loop to find this.

In [ ]:
top_bike = ['some hour', 0]
top_ped = ['some hour', 0]

for hour, traffic in traffic_by_hour_bp.items():

    if traffic['bikes'] > top_bike[1]:
        top_bike[0] = hour
        top_bike[1] = traffic['bikes']
    else:
        pass
    
    if traffic['pedestrians'] > top_ped[1]:
        top_ped[0] = hour
        top_ped[1] = traffic['pedestrians']
    else:
        pass
    
print("The hour with the most bike traffic is " + top_bike[0] + ", with " + str(top_bike[1]) + " bikes!")
print("The hour with the most pedestrian traffic is " + top_ped[0] + ", with " + str(top_ped[1]) + " pedestrians!")

    

### Going further with Question 3
- Challenge #3: Were there more bikes or pedestrians using the Burke Gilman trail in 2019?

## Question 4: What is the busiest hour for bikes vs. peds AND northbound vs. southbound?

If we want, we can break our data down by bikes vs. peds AND northbound vs. southbound. This is useful because it helps us understand how many people are using the Burke Gilman trail for daily commuting, not just for pleasure trips. 

Splitting bike and pedestrian traffic by northboud/southbound is actually almost as easy as lumping them together. We just make our dictionary-in-a-dictionary a little more complex.


In [ ]:
#create a new empty dictionary
traffic_by_hour_bpd = {}

for hour_data in raw_data:
    
    traffic_hour = hour_data['date'][11:19]
    
    if traffic_hour not in traffic_by_hour_bpd.keys():

        traffic_by_hour_bpd[traffic_hour] = {'bikes north' : 0, 'pedestrians north' : 0, 
                                            'bikes south' : 0, 'pedestrians south' : 0}  
    else:
        pass

pprint(traffic_by_hour_bpd)

And now instead of adding bike_north + bike_south etc together, we store each of those counts in their own key/value bucket.

In [ ]:
for hour_data in raw_data:
    
    traffic_hour = hour_data['date'][11:19]
    
    traffic_by_hour_bpd[traffic_hour]['bikes north'] += int(hour_data['bike_north']) #store count in bike_north
    traffic_by_hour_bpd[traffic_hour]['bikes south'] += int(hour_data['bike_south']) #store count in bike_south
    
    traffic_by_hour_bpd[traffic_hour]['pedestrians north'] += int(hour_data['ped_north'])
    traffic_by_hour_bpd[traffic_hour]['pedestrians south'] += int(hour_data['ped_south'])

pprint(traffic_by_hour_bpd)

### Answering question 4
I will leave it up to you to find the answer to any of these questions (you already know how to do it!)
- What is the busiest hour for northbound bike traffic? For southbound bike traffic?
- What is the busiest hour for northbound pedestrian traffic? For southbound pedestrian traffic?

If you don't feel like writing code to answer these questions right now, that's okay. Below, I will show you how to export this data in a format that can be used to easily create timeseries graphs in a spreadsheet. It will be much easier to answer these questions with a graph than by looking at rows of numbers!

## Exporting the bike/ped north/south dataset for visualization

Now we have some nice rich data that we can graph in a spreadsheet program like Google Sheets or Microsoft Excel. 

But to do that, we will need to convert the data to CSV format and export it into a file. We ideally want a format that has a row for each hour, and columns like
``hour_of_day | bikes_north | bikes_south | pedestrians_north | pedestrians_south``
 
The best way to do this is to convert our data from a nested dictionary to a nested list (a list-of-lists!), where each sub-list (which will be a row in our CSV file), contains the values in a consistent order, like:

``[hour_of_day, bike_north, bike_south, ped_north, ped_south]``

In [ ]:
#create a new empty 'master list'
traffic_by_hour_mode_direction = []

#for each hour in our traffic-by-hour-mode-direction dictionary...
for hour, counts in traffic_by_hour_bpd.items():
    
    #...create a new sub-list that will store the hour, bike, and pedestrian data in a consistent order
    list_element = [] #new empty sub-list
    list_element.append(hour) #add the hour in, e.g. ['06:00:00']
    list_element.append(counts['bikes north']) #append bike north count, e.g. ['06:00:00', 484]
    list_element.append(counts['bikes south']) #append bike south count, e.g. ['06:00:00', 484, 82]
    list_element.append(counts['pedestrians north']) # etc...
    list_element.append(counts['pedestrians south']) # etc... 
    
    traffic_by_hour_mode_direction.append(list_element) #add this list to the end of our growing master list
    
    

In [ ]:
pprint(traffic_by_hour_mode_direction)

This worked! BUT something odd happened. Can you see it? For some reason, our '00:00:00' hour got moved to the end of the list. 

This happened because Python dictionaries do not reliably preserve the order of items! So even though the data looked like it was in the right order when we had it in the dictionary, when we created our new master list from ``traffic_by_hour_bpd`` it ended up a little out of order. 

Fortunately, unlike dictionaries, lists-of-lists are easy to sort! Especially if you want to sort by the first item in each sub-list, which we do!

In [ ]:
#sort the list in 'alphabetical order' by the first sub-item, which is the hour stamp
traffic_by_hour_mode_direction.sort() 

In [ ]:
pprint(traffic_by_hour_mode_direction)

Now that we have our data in a list, it's easy to export to a CSV file!

In [ ]:
import csv

In [ ]:
with open('bg_traffic_bike_ped_2019.csv', 'w', encoding='utf-8') as f:
    writer = csv.writer(f)
    #write a header row
    writer.writerow(('hour of the day', 
                     'northbound bikes', 
                     'southbound bikes', 
                     'northbound pedestrians',
                     'southbound pedestrians'))
    
    for i in traffic_by_hour_mode_direction:
        writer.writerow((i[0], i[1], i[2], i[3], i[4]))

## Question 5 (Grand challenge!): What day of the week is busiest on the Burke Gilman?

If we really want to understand whether people use the Burke Gilman for commuting, rather than just for recreation, we probably need to know what days of the week are busiest. 

We can answer this question using the same basic approach we used to answer the previous questions, but we will need a little bit of help. Since days of the week occur on different calendar days every year, we need a way to find out, for any given date in 2019, what day of the week that date fell on.

Python has a couple of tools that can help us with this:
- ``parse`` is a function from the ``dateutil`` library that can turn a date that is formatted as a string into a "datetime object", which is a format that Python understands as a real date, not just a series of characters.
- ``weekday`` is a function from the ``datetime`` library that can tell you, for any valid datetime object, what day of the week that date fell on.

In [ ]:
from datetime import datetime #yes, I know this seems redundant
from dateutil import parser

Let's try out formatting one of our strings as a datetime object.

In [ ]:
#here's a random date from our dataset, in its native string format
print(raw_data[234]['date'])

In [ ]:
#now let's use the parse function to turn that into datetime format, which Python can work with
parser.parse(raw_data[234]['date'])

That worked! We can save this datetime object in its own variable. That will let us do things with it (like find out what day of the week January 10th, 2019 was).

In [ ]:
my_date = parser.parse(raw_data[234]['date'])
print(my_date)
print(type(my_date))

Now that we know how to parse strings into datetime objects, we can use the ``datetime.weekday()`` function to find out what day of the week it was. This function will return a value between 0 (Monday) and 6 (Sunday) for any valid date you give it.

In [ ]:
my_date.weekday() #starts at 0!

Looks like January 10th was a Thursday (you can check your calendar to confirm this).

### Answering Question 5
Now that you know how to find out the day of the week for any date in our ``raw_data`` dataset, you should be able to find out which weekday or weekend day sees the most traffic on the Burke Gilman. You'll be able to use loops, "if" statements, and dictionaries to calculate the totals for each day. 

Before you start, think for a minute: what do you think the answer will be, based on your own experience or your prior knowledge? Do you think the Burke Gilman is used more on weekdays or weekends? Why? Are some weekdays busier than others? Why?


# Congratulations! 

You have now mastered manipulating timeseries data in Python. There are plenty of other techniques, tools, and time-saving tricks that you can learn to build on these skills, but many data scientists who use Python every day do this kind of work using the same basic approach you just learned.

## Challenges: going further

Here are some additional questions that you now have the tools you need to answer, based on what you've done today:

- what day of the week is busiest for bikes? Is it the same as the busiest day for pedestrians?
- what month of the year is busiest? (aka do Seattlites really like to ride in the rain?)
- has the Burke Gilman gotten busier over time? (the dataset we have goes back to 2014!)
- do fewer people commute on the Burke Gilman when it's cold out? (hint: try combining this dataset with the [dataset on road temperature over time](https://data.seattle.gov/Public-Safety/Road-Weather-Information-Stations/egc4-d24i/data)
- do more people commute into Seattle in the mornings by bike on the Burke Gilman, or on the [the Mountain to Sound Trail](https://data.seattle.gov/Transportation/MTS-Trail-west-of-I-90-Bridge-Bicycle-and-Pedestri/u38e-ybnc)?

## Other data.seattle.gov datasets that you can do timeseries with...
- Fremont bridge bicycle counter: https://data.seattle.gov/Transportation/Fremont-Bridge-Bicycle-Counter/65db-xm6k
- Spokane Street bridge bicycle counter: https://data.seattle.gov/Transportation/Spokane-St-Bridge-Bicycle-Counter/upms-nr8w
- Mountain to Sound trail bicycle + pedestrian counter: https://data.seattle.gov/Transportation/MTS-Trail-west-of-I-90-Bridge-Bicycle-and-Pedestri/u38e-ybnc


## More complex datasets

- https://data.seattle.gov/Public-Safety/Terry-Stops/28ny-9ts8
- https://data.seattle.gov/Permitting/Building-Permits/76t5-zqzr